# QuantumEdge Tutorial 2: Quantum vs Classical Portfolio Optimization

**Comparative Analysis of VQE, QAOA, and Mean-Variance Optimization**

This notebook demonstrates the key differences between quantum-inspired portfolio optimization algorithms and classical mean-variance optimization. We'll compare their performance, characteristics, and use cases using real market data.

## Overview

- **Classical Mean-Variance**: Markowitz optimization for continuous allocation
- **VQE (Variational Quantum Eigensolver)**: Finds eigenportfolios with quantum-inspired circuits
- **QAOA (Quantum Approximate Optimization Algorithm)**: Discrete asset selection with cardinality constraints

---

## 1. Setup and Data Preparation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time
from datetime import datetime, timedelta
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("QuantumEdge Comparative Analysis")
print("=" * 40)

In [ ]:
# QuantumEdge API configuration
API_BASE_URL = "http://localhost:8000/api/v1"

def check_api_health():
    """Check if QuantumEdge API is running"""
    try:
        response = requests.get(f"{API_BASE_URL.replace('/api/v1', '')}/health")
        if response.status_code == 200:
            print("✅ QuantumEdge API is healthy")
            return True
        else:
            print(f"❌ API health check failed: {response.status_code}")
            return False
    except Exception as e:
        print(f"❌ Cannot connect to API: {e}")
        print("Make sure to run: cd /path/to/QuantumEdge && python -m src.api.main")
        return False

check_api_health()

## 2. Market Data Collection

We'll use a diversified portfolio of technology stocks to compare optimization approaches.

In [ ]:
# Define our comparison portfolio
SYMBOLS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA']
START_DATE = '2022-01-01'
END_DATE = '2024-01-01'

print(f"Fetching data for: {', '.join(SYMBOLS)}")
print(f"Period: {START_DATE} to {END_DATE}")

# Download market data
data = yf.download(SYMBOLS, start=START_DATE, end=END_DATE)['Adj Close']
returns = data.pct_change().dropna()

print(f"\nData shape: {data.shape}")
print(f"Returns shape: {returns.shape}")

# Display basic statistics
print("\nAnnualized Returns and Volatility:")
annual_returns = returns.mean() * 252
annual_volatility = returns.std() * np.sqrt(252)

stats_df = pd.DataFrame({
    'Annual Return': annual_returns,
    'Annual Volatility': annual_volatility,
    'Sharpe Ratio': annual_returns / annual_volatility
})
print(stats_df.round(4))

In [ ]:
# Prepare inputs for optimization
expected_returns = annual_returns.values
cov_matrix = returns.cov().values * 252  # Annualized covariance
returns_data = returns.values  # For CVaR and other advanced objectives

print("Expected Returns:")
for i, symbol in enumerate(SYMBOLS):
    print(f"{symbol}: {expected_returns[i]:.4f}")

print(f"\nCovariance Matrix Shape: {cov_matrix.shape}")
print(f"Returns Data Shape: {returns_data.shape}")

## 3. Classical Mean-Variance Optimization

Let's start with traditional Markowitz optimization using multiple objectives.

In [ ]:
def optimize_mean_variance(objective='maximize_sharpe', **kwargs):
    """Run mean-variance optimization with QuantumEdge API"""
    
    payload = {
        "expected_returns": expected_returns.tolist(),
        "covariance_matrix": cov_matrix.tolist(),
        "objective": objective,
        "risk_aversion": 1.0,
        **kwargs
    }
    
    # Add historical returns for advanced objectives
    if objective in ['minimize_cvar', 'maximize_sortino', 'maximize_calmar']:
        payload["returns_data"] = returns_data.tolist()
        if objective == 'minimize_cvar':
            payload["cvar_confidence"] = 0.05
        elif objective == 'maximize_calmar':
            payload["lookback_periods"] = 252
    
    response = requests.post(f"{API_BASE_URL}/optimize/mean-variance", json=payload)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Optimization failed: {response.status_code}")
        print(response.text)
        return None

# Test different mean-variance objectives
mv_objectives = ['maximize_sharpe', 'minimize_variance', 'maximize_return', 'minimize_cvar']
mv_results = {}

print("Running Mean-Variance Optimizations...")
for obj in mv_objectives:
    print(f"  • {obj.replace('_', ' ').title()}")
    result = optimize_mean_variance(obj)
    if result and result['success']:
        mv_results[obj] = result
        time.sleep(0.5)  # Rate limiting
    else:
        print(f"    ❌ Failed")

print(f"\nCompleted {len(mv_results)} mean-variance optimizations")

## 4. Quantum-Inspired VQE Optimization

VQE finds eigenportfolios by minimizing the portfolio variance using quantum-inspired variational circuits.

In [ ]:
def optimize_vqe(depth=3, num_eigenportfolios=3):
    """Run VQE optimization with QuantumEdge API"""
    
    payload = {
        "covariance_matrix": cov_matrix.tolist(),
        "depth": depth,
        "optimizer": "COBYLA",
        "max_iterations": 100,
        "num_eigenportfolios": num_eigenportfolios,
        "num_random_starts": 5
    }
    
    response = requests.post(f"{API_BASE_URL}/quantum/vqe", json=payload)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"VQE optimization failed: {response.status_code}")
        print(response.text)
        return None

print("Running VQE Optimization...")
vqe_result = optimize_vqe(depth=3, num_eigenportfolios=1)

if vqe_result and vqe_result['success']:
    print("✅ VQE optimization completed")
    print(f"   Solve time: {vqe_result['solve_time']:.3f}s")
    print(f"   Eigenvalue: {vqe_result['portfolio']['objective_value']:.6f}")
else:
    print("❌ VQE optimization failed")
    vqe_result = None

## 5. Quantum-Inspired QAOA Optimization

QAOA solves discrete portfolio selection problems with cardinality constraints.

In [ ]:
def optimize_qaoa(cardinality=3, num_layers=3):
    """Run QAOA optimization with QuantumEdge API"""
    
    payload = {
        "expected_returns": expected_returns.tolist(),
        "covariance_matrix": cov_matrix.tolist(),
        "risk_aversion": 1.0,
        "num_layers": num_layers,
        "optimizer": "COBYLA",
        "max_iterations": 100,
        "cardinality_constraint": cardinality
    }
    
    response = requests.post(f"{API_BASE_URL}/quantum/qaoa", json=payload)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"QAOA optimization failed: {response.status_code}")
        print(response.text)
        return None

print("Running QAOA Optimization...")
qaoa_result = optimize_qaoa(cardinality=3, num_layers=3)

if qaoa_result and qaoa_result['success']:
    print("✅ QAOA optimization completed")
    print(f"   Solve time: {qaoa_result['solve_time']:.3f}s")
    print(f"   Objective value: {qaoa_result['portfolio']['objective_value']:.6f}")
    
    # Count selected assets
    weights = np.array(qaoa_result['portfolio']['weights'])
    selected_assets = np.sum(weights > 0.01)
    print(f"   Selected assets: {selected_assets}/{len(SYMBOLS)}")
else:
    print("❌ QAOA optimization failed")
    qaoa_result = None

## 6. Comparative Analysis

Now let's compare the results across all optimization approaches.

In [ ]:
# Collect all results for comparison
all_results = {}

# Add mean-variance results
for obj, result in mv_results.items():
    all_results[f"MV_{obj}"] = result

# Add quantum results
if vqe_result:
    all_results["VQE"] = vqe_result
if qaoa_result:
    all_results["QAOA"] = qaoa_result

print(f"Comparing {len(all_results)} optimization strategies:")
for name in all_results.keys():
    print(f"  • {name}")

In [ ]:
# Create comparison DataFrame
comparison_data = []

for name, result in all_results.items():
    if result['success'] and result.get('portfolio'):
        portfolio = result['portfolio']
        weights = np.array(portfolio['weights'])
        
        # Calculate additional metrics
        num_assets = np.sum(weights > 0.01)  # Assets with >1% allocation
        concentration = np.max(weights)  # Largest weight
        diversification = 1 / np.sum(weights**2)  # Effective number of assets
        
        comparison_data.append({
            'Strategy': name,
            'Expected Return': portfolio['expected_return'],
            'Volatility': portfolio['volatility'],
            'Sharpe Ratio': portfolio.get('sharpe_ratio', 0),
            'Assets Used': num_assets,
            'Max Weight': concentration,
            'Diversification': diversification,
            'Solve Time': result['solve_time']
        })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.round(4)

print("\nOptimization Strategy Comparison:")
print("=" * 50)
print(comparison_df.to_string(index=False))

## 7. Visualization and Analysis

In [ ]:
# Create portfolio allocation comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

plot_idx = 0
for i, (name, result) in enumerate(all_results.items()):
    if plot_idx >= 6:  # Limit to 6 plots
        break
        
    if result['success'] and result.get('portfolio'):
        weights = np.array(result['portfolio']['weights'])
        
        # Create pie chart for non-zero weights
        non_zero_mask = weights > 0.001
        if np.any(non_zero_mask):
            labels = [SYMBOLS[i] for i in range(len(SYMBOLS)) if non_zero_mask[i]]
            sizes = weights[non_zero_mask]
            
            axes[plot_idx].pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
            axes[plot_idx].set_title(f'{name}\nSharpe: {result["portfolio"].get("sharpe_ratio", 0):.3f}')
        
        plot_idx += 1

# Hide unused subplots
for i in range(plot_idx, 6):
    axes[i].set_visible(False)

plt.tight_layout()
plt.suptitle('Portfolio Allocation Comparison', fontsize=16, y=1.02)
plt.show()

In [ ]:
# Risk-Return Scatter Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Risk-Return Space
colors = plt.cm.Set3(np.linspace(0, 1, len(comparison_df)))

for i, row in comparison_df.iterrows():
    ax1.scatter(row['Volatility'], row['Expected Return'], 
               s=200, alpha=0.7, color=colors[i], 
               label=row['Strategy'])
    
    # Add text annotation
    ax1.annotate(row['Strategy'], 
                (row['Volatility'], row['Expected Return']),
                xytext=(5, 5), textcoords='offset points',
                fontsize=8, alpha=0.8)

ax1.set_xlabel('Volatility (Risk)')
ax1.set_ylabel('Expected Return')
ax1.set_title('Risk-Return Profile Comparison')
ax1.grid(True, alpha=0.3)

# Plot 2: Sharpe Ratio Comparison
strategies = comparison_df['Strategy']
sharpe_ratios = comparison_df['Sharpe Ratio']

bars = ax2.bar(range(len(strategies)), sharpe_ratios, color=colors)
ax2.set_xlabel('Strategy')
ax2.set_ylabel('Sharpe Ratio')
ax2.set_title('Sharpe Ratio Comparison')
ax2.set_xticks(range(len(strategies)))
ax2.set_xticklabels(strategies, rotation=45, ha='right')
ax2.grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars, sharpe_ratios):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{value:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Diversification and Concentration Analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Number of Assets vs Diversification Score
ax1.scatter(comparison_df['Assets Used'], comparison_df['Diversification'], 
           s=200, alpha=0.7, c=comparison_df['Sharpe Ratio'], 
           cmap='viridis')

for i, row in comparison_df.iterrows():
    ax1.annotate(row['Strategy'], 
                (row['Assets Used'], row['Diversification']),
                xytext=(5, 5), textcoords='offset points',
                fontsize=8, alpha=0.8)

ax1.set_xlabel('Number of Assets Used')
ax1.set_ylabel('Diversification Score')
ax1.set_title('Portfolio Diversification Analysis')
ax1.grid(True, alpha=0.3)

# Add colorbar
cbar = plt.colorbar(ax1.collections[0], ax=ax1)
cbar.set_label('Sharpe Ratio')

# Plot 2: Concentration Risk
strategies = comparison_df['Strategy']
max_weights = comparison_df['Max Weight']

bars = ax2.bar(range(len(strategies)), max_weights, 
               color=plt.cm.Reds(max_weights / max_weights.max()))
ax2.set_xlabel('Strategy')
ax2.set_ylabel('Maximum Weight')
ax2.set_title('Concentration Risk (Largest Single Position)')
ax2.set_xticks(range(len(strategies)))
ax2.set_xticklabels(strategies, rotation=45, ha='right')
ax2.grid(True, alpha=0.3)

# Add horizontal line at 20% (common concentration limit)
ax2.axhline(y=0.2, color='red', linestyle='--', alpha=0.7, label='20% Limit')
ax2.legend()

plt.tight_layout()
plt.show()

## 8. Performance Analysis and Insights

In [ ]:
# Calculate correlation between strategies
weights_matrix = []
strategy_names = []

for name, result in all_results.items():
    if result['success'] and result.get('portfolio'):
        weights_matrix.append(result['portfolio']['weights'])
        strategy_names.append(name)

if len(weights_matrix) > 1:
    weights_df = pd.DataFrame(weights_matrix, 
                             index=strategy_names, 
                             columns=SYMBOLS)
    
    # Calculate correlation matrix
    correlation_matrix = weights_df.T.corr()
    
    # Plot correlation heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                square=True, linewidths=0.5)
    plt.title('Strategy Correlation Matrix\n(Based on Portfolio Weights)')
    plt.tight_layout()
    plt.show()
    
    print("Portfolio Weight Allocations:")
    print("=" * 30)
    print(weights_df.round(4))

In [ ]:
# Computational Performance Analysis
solve_times = comparison_df.set_index('Strategy')['Solve Time']
sharpe_ratios = comparison_df.set_index('Strategy')['Sharpe Ratio']

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Solve time comparison
solve_times.plot(kind='bar', ax=ax1, color='skyblue', alpha=0.7)
ax1.set_title('Computational Performance (Solve Time)')
ax1.set_ylabel('Solve Time (seconds)')
ax1.set_xlabel('Strategy')
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Efficiency analysis (Sharpe per second)
efficiency = sharpe_ratios / solve_times
efficiency.plot(kind='bar', ax=ax2, color='lightgreen', alpha=0.7)
ax2.set_title('Optimization Efficiency (Sharpe Ratio per Second)')
ax2.set_ylabel('Sharpe Ratio / Solve Time')
ax2.set_xlabel('Strategy')
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\nComputational Efficiency Analysis:")
print("=" * 35)
efficiency_df = pd.DataFrame({
    'Solve Time': solve_times,
    'Sharpe Ratio': sharpe_ratios,
    'Efficiency': efficiency
}).round(4)
print(efficiency_df)

## 9. Key Insights and Recommendations

Based on our comparative analysis, here are the key findings:

In [ ]:
# Generate insights based on the results
insights = []

# Best Sharpe ratio
best_sharpe_idx = comparison_df['Sharpe Ratio'].idxmax()
best_sharpe_strategy = comparison_df.loc[best_sharpe_idx, 'Strategy']
best_sharpe_value = comparison_df.loc[best_sharpe_idx, 'Sharpe Ratio']
insights.append(f"🏆 Best Risk-Adjusted Returns: {best_sharpe_strategy} (Sharpe: {best_sharpe_value:.3f})")

# Most diversified
most_diversified_idx = comparison_df['Diversification'].idxmax()
most_diversified_strategy = comparison_df.loc[most_diversified_idx, 'Strategy']
diversification_score = comparison_df.loc[most_diversified_idx, 'Diversification']
insights.append(f"🌐 Most Diversified: {most_diversified_strategy} (Score: {diversification_score:.2f})")

# Fastest optimization
fastest_idx = comparison_df['Solve Time'].idxmin()
fastest_strategy = comparison_df.loc[fastest_idx, 'Strategy']
fastest_time = comparison_df.loc[fastest_idx, 'Solve Time']
insights.append(f"⚡ Fastest Optimization: {fastest_strategy} ({fastest_time:.3f}s)")

# Most efficient
if len(efficiency_df) > 0:
    most_efficient_strategy = efficiency_df['Efficiency'].idxmax()
    efficiency_value = efficiency_df.loc[most_efficient_strategy, 'Efficiency']
    insights.append(f"⚙️ Most Efficient: {most_efficient_strategy} (Efficiency: {efficiency_value:.3f})")

# Quantum vs Classical comparison
quantum_strategies = [name for name in comparison_df['Strategy'] if name in ['VQE', 'QAOA']]
classical_strategies = [name for name in comparison_df['Strategy'] if 'MV_' in name]

if quantum_strategies and classical_strategies:
    quantum_avg_sharpe = comparison_df[comparison_df['Strategy'].isin(quantum_strategies)]['Sharpe Ratio'].mean()
    classical_avg_sharpe = comparison_df[comparison_df['Strategy'].isin(classical_strategies)]['Sharpe Ratio'].mean()
    
    if quantum_avg_sharpe > classical_avg_sharpe:
        insights.append(f"🔬 Quantum approaches show {((quantum_avg_sharpe/classical_avg_sharpe - 1) * 100):.1f}% better average Sharpe ratio")
    else:
        insights.append(f"📊 Classical approaches show {((classical_avg_sharpe/quantum_avg_sharpe - 1) * 100):.1f}% better average Sharpe ratio")

print("\n" + "="*60)
print("KEY INSIGHTS AND RECOMMENDATIONS")
print("="*60)

for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

print("\n" + "="*60)
print("STRATEGY RECOMMENDATIONS:")
print("="*60)

recommendations = [
    "🎯 Mean-Variance (Sharpe): Best for traditional risk-adjusted optimization",
    "🛡️ Mean-Variance (CVaR): Ideal for tail risk management and downside protection",
    "⚡ VQE: Suitable for finding minimum variance eigenportfolios with quantum inspiration",
    "🎲 QAOA: Perfect for discrete asset selection with cardinality constraints",
    "🏃 Classical methods: Generally faster for continuous optimization problems",
    "🔬 Quantum methods: Better for combinatorial problems and alternative risk measures"
]

for recommendation in recommendations:
    print(f"• {recommendation}")

## 10. Conclusion

This comparative analysis demonstrates the strengths and characteristics of different portfolio optimization approaches:

### **Classical Mean-Variance Optimization**
- **Strengths**: Fast, reliable, well-understood mathematical foundation
- **Best for**: Continuous weight optimization, traditional risk metrics
- **Advanced objectives**: CVaR, Sortino, and Calmar ratios provide sophisticated risk management

### **VQE (Variational Quantum Eigensolver)**
- **Strengths**: Finds eigenportfolios, quantum-inspired optimization
- **Best for**: Minimum variance portfolios, alternative to traditional mean-variance
- **Characteristics**: Focuses on risk minimization rather than return maximization

### **QAOA (Quantum Approximate Optimization Algorithm)**
- **Strengths**: Handles discrete selection, cardinality constraints naturally
- **Best for**: Asset selection problems, ESG screening, regulatory constraints
- **Characteristics**: Produces sparse portfolios with limited number of holdings

### **When to Use Each Approach**

1. **Traditional portfolio management** → Mean-Variance with Sharpe optimization
2. **Risk management focus** → Mean-Variance with CVaR or Sortino objectives
3. **Minimum variance seeking** → VQE for eigenportfolio approach
4. **Asset selection with limits** → QAOA for discrete optimization
5. **Regulatory constraints** → QAOA for complex selection rules

The choice of optimization method should align with your specific investment objectives, constraints, and risk preferences. QuantumEdge provides the flexibility to experiment with all approaches and find the most suitable solution for your portfolio management needs.

---

**Next Steps**: 
- Try `03_backtesting_strategies.ipynb` to see how these optimizations perform over time
- Explore `04_advanced_objectives.ipynb` for deep dive into CVaR, Calmar, and Sortino optimization
- Use `05_risk_analysis.ipynb` for comprehensive risk metric analysis